In [1]:
import json
path = "wrong_op/20200621_Python_wrong_binary_operator_datasets_train.jsontxt-00003-of-00004"
with open(path, "r") as f:
    lines = f.readlines()
data = [json.loads(line) for line in lines]

In [3]:
print(data[0]["function"])

def url(self, name):
        if STATIC_URL is None:
            url = super(VersionedS3BotoStorage, self).url(name)
            return "{}?{}".format(url, settings.COMMIT_SHA)
        else:
            return "{}{}?{}".format(STATIC_URL, name, settings.COMMIT_SHA)


In [73]:
import os
import re

In [143]:
dir = "./wrong_op"
files = [file for file in os.listdir(dir) if os.path.isfile(os.path.join(dir, file))]
files = [file for file in files if "eval" in file]

In [144]:
datadics = []
for file in files:
    with open(os.path.join(dir, file), "r") as f:
        for line in f.readlines():
            datadics.append(json.loads(line))

In [145]:
hashdic = {}
for dic in datadics:
    info = dic["info"]
    sp = info.split(" ")
    key = " ".join(sp[:2])
    key = "/".join(key.split("/")[:-1])
    label = dic["label"] != "Correct"
    curdic = hashdic.get(key, {})
    curdic[label] = {"function": dic["function"]}
    curdic[label]["info"] = [None, None]
    if label:
        sp = re.findall(r"`(.*?)`", info)
        # print(sp)
        assert(len(sp) == 2)
        curdic[label]["info"] = sp
    hashdic[key] = curdic

In [146]:
len(hashdic)

125134

In [147]:
keys = list(hashdic.keys())
for key in keys:
    if len(hashdic[key]) < 2:
        del hashdic[key]

In [43]:
import os
import sys
from tqdm import tqdm
import random
import pickle
import glob
import javalang
import tree_sitter
from tree_sitter import Language, Parser

In [44]:
Language.build_library(
  # Store the library in the `build` directory
  'build/py.so',

  # Include one or more languages
  [
    './tree-sitter-python'
  ]
)
PY_LANGUAGE = Language('build/py.so', 'python')
parser = Parser()
parser.set_language(PY_LANGUAGE)

In [58]:
def node2tokens(node, code_bytes):
    if node.type == "comment":
        return []
    if not node.children:
        return [code_bytes[node.start_byte:node.end_byte].decode()]
    tokens = []
    for child in node.children:
        a = node2tokens(child, code_bytes)
        tokens += a
    return tokens

def gettokens(parser, code):
    code_bytes = code.encode("utf-8")
    return node2tokens(parser.parse(code_bytes).root_node, code_bytes)

In [79]:
from tqdm import tqdm

In [148]:
for key in tqdm(hashdic.keys()):
    dic1 = hashdic[key][0]
    dic2 = hashdic[key][1]
    dic1["label"] = 0
    dic2["label"] = 1
    tokens1 = gettokens(parser, dic1["function"])
    tokens2 = gettokens(parser, dic2["function"])
    for i in range(len(tokens1)):
        if tokens1[i] != tokens2[i]:
            break
    spanl = 1
    if error[0] == "is not" and error[1] == "is":
        i -= 1
    elif error[0] == "is" and error[1] == "is not":
        i -= 1
    dic1["idx"] = 0
    dic1["span"] = (0, 0)
    dic1["tokens"] = tokens1
    dic2["idx"] = i
    spanl = len(dic2['info'][1].split(" "))
    dic2["span"] = (i, i + spanl)
    dic2["tokens"] = tokens2

100%|██████████| 125134/125134 [01:45<00:00, 1181.28it/s]


In [149]:
for key in keys[0:20]:
    print(" ".join(hashdic[key][0]["tokens"]))
    print(hashdic[key][1]["span"])
    print(" ".join(hashdic[key][1]["tokens"]))
    print(hashdic[key][1]["tokens"][hashdic[key][1]["span"][0]])
    print()

@ staticmethod def filter_value ( key , value ) : xpath = " " if isinstance ( value , str ) : if ' ' in value : value = value . replace ( ' ' , ' ' ) xpath = ' ' % ( key , value ) else : xpath = ' ' % ( key , value ) return xpath
(25, 26)
@ staticmethod def filter_value ( key , value ) : xpath = " " if isinstance ( value , str ) : if ' ' <= value : value = value . replace ( ' ' , ' ' ) xpath = ' ' % ( key , value ) else : xpath = ' ' % ( key , value ) return xpath
<=

@ staticmethod def xpath ( filter = { } ) : xpath = " " if filter : filter_list = [ ] for ( key , value ) in filter . items ( ) : if key == ' ' : key = ' ' else : key = ' ' + key if isinstance ( value , str ) : filter_list . append ( XpathFilter . filter_value ( key , value ) ) elif isinstance ( value , list ) : stmt = ' ' . join ( [ XpathFilter . filter_value ( key , str ( val ) ) for val in value ] ) filter_list . append ( stmt ) if filter_list : xpath = ' ' . join ( filter_list ) xpath = ' ' + xpath + ' ' return xpath


In [113]:
def mask_special_value(token):
    # print(token)
    if len(token) == 0:
        return token
    if token[0] in "0123456789":
        if 'e' in token.lower() or '.' in token:
            return "<fp>"
        else:
            return "<int>"
    elif token[0] == '"' and token[-1] == '"':
        return "<str>"
    elif token[0] == "'" and token[-1] == "'":
        return "<str>"
    else:
        return token

In [150]:
function, raw, norm, idx, span, label = [], [], [], [], [], []
for key in hashdic:
    for subkey in hashdic[key]:
        dic1 = hashdic[key][subkey]
        function.append(dic1["function"])
        raw.append(dic1["tokens"])
        # print(dic1["tokens"])
        norm.append(list(filter(lambda t: len(t) > 0, map(mask_special_value, dic1["tokens"]))))
        idx.append(dic1["idx"])
        span.append(dic1["span"])
        label.append(dic1["label"])

In [141]:
idx[0]

0

In [151]:
with open("wrong_op/test.pkl", "wb") as f:
    pickle.dump({"raw": raw,
                 "norm": norm,
                 "idx": idx,
                 "span": span,
                 "label": label,
                 "function": function}, f)

In [3]:
import pickle
with open("wrong_op/valid.pkl", "rb") as f:
    data = pickle.load(f)
print(len(data["raw"]))
# print(sum(data["label"]))

49460


In [7]:
len(data["norm"][0])

14

In [10]:
sum (list(map(len, (data["raw"])))) / len(data["norm"])

122.71951071572988

In [8]:
data["norm"][0]

['def',
 '__virtual__',
 '(',
 ')',
 ':',
 'if',
 'HAS_RAVEN',
 'is',
 'True',
 ':',
 'return',
 '__virtualname__',
 'return',
 'False']